In [1]:
import pandas as pd 
import numpy as np
import lightgbm as lgb
import sklearn

In [ ]:
train = pd.read_csv('cleaning_train.csv')
tr = train.copy()

In [ ]:
train['TotalScore'] = train['HomeScoreBeforePlay'] + train['VisitorScoreBeforePlay']

In [ ]:
train['GameClock'] = int(train['GameClock'][0][:2])+float(train['GameClock'][0][3:5])/60

In [ ]:
train['CumGameClock'] = train['GameClock'] / 15

In [ ]:
train['OffensePersonnel'].unique()

In [ ]:
pos = set()
def personnel(a):
    global pos
    temp = a.split(',')
    for i in range(len(temp)):
        x = temp[i].strip()
        pos.add(x[2:4])
        
train['OffensePersonnel'].map(personnel)
df = pd.DataFrame(columns=pos)
for i in df.columns:
    df[i] = [0]*len(train)

In [ ]:
cols_to_drop = [
    'Location', 'GameId', 'PlayId', 'PlayerCollegeName', 
]

In [ ]:
pos

In [ ]:
df = pd.DataFrame(columns=pos)
for i in df.columns:
    df[i] = [0]*len(train)

In [ ]:
%%time
for i in range(len(train)):
    temp = train['OffensePersonnel'][i].split(',')
    for j in range(len(temp)):
        x = temp[j].strip()
#         print(x[2:4])
        df[x[2:4]][i] = int(x[0])

In [ ]:
df.to_csv('offensepersonnel.csv', index=False)

In [ ]:
train.columns

In [ ]:
train.iloc[0,:]

In [ ]:
train.describe()

In [ ]:
train['NflIdRusher']

In [ ]:
%%time
for i in range(len(train)):
    if train['Team'][i] == 'away':
        h[i] = train['VisitorTeamAbbr'][i]
        vt[i] = train['HomeTeamAbbr'][i]
    else:
        h[i] = train['HomeTeamAbbr'][i]
        vt[i] = train['VisitorTeamAbbr'][i]

In [ ]:
train['VsTeam'] = vt
train['HomeTeam'] = h

In [ ]:
h = [0] * len(train)
vt = [0] * len(train)

In [ ]:
off = pd.read_csv('offensepersonnel.csv')

In [ ]:
off.columns

In [ ]:
a = list(off.columns)

In [ ]:
for i in range(len(a)):
    a[i] = 'OFF_'+a[i]

In [ ]:
a

In [ ]:
off.columns = a

In [ ]:
off.to_csv('offensepersonnel.csv', index=False)

In [ ]:
train['StadiumType'].unique()

In [ ]:
train.iloc[0,:]

In [ ]:
offense = [0] * len(train)

for i in range(len(train)):
    if train['PossessionTeam'][i] == train['HomeTeam'][i]:
        offense[i] = 1

In [ ]:
train['IsOffense'] = offense

In [ ]:

train = train.drop(['HomeTeamAbbr', 'VisitorTeamAbbr'], axis=1)

In [ ]:
train['PlayerBirthDate'] = age

In [ ]:
age = [0]*len(train)
for i in range(len(train)):
    age[i] = 2019 - int(train['PlayerBirthDate'][i][-4:])

In [ ]:
ctd = [
    'GameId', 'PlayId', 'OffensePersonnel', 'DefensePersonnel',
    'PlayDirection', 'PlayerCollegeName', 'Stadium', 'WindDirection', 'Location',
    'DisplayName', 'Team'
    
]

In [ ]:
rish = [0]*len(train)
for i in range(len(train)):
    if train['NflId'][i] == train['NflIdRusher'][i]:
        rish[i] = 1

train['NflIdRusher'] = rish

In [ ]:
train['Position'].unique()

In [ ]:
train['GameWeather'].replace(np.nan, 'NAN').map(low).unique()

In [ ]:
def low(a):
    a = a.lower()
    return a

In [ ]:
weather = {
    'sun': 1,
    'cloud': 0,
    'clear': 1,
    'rain': -1,
    'snow': -1    
}

def weather(a):
    if ('sun' in a) or ('clear' in a) or ('fair' in a):
        return 1
    elif ('rain' in a) or ('snow' in a) or ('shower' in a):
        return -1
    elif ('cloud' in a) or ('coudy' in a) or ('clouid' in a):
        return 0.5
    elif ('cold' in a) or ('hazy' in a) or ('overcast' in a):
        return -0.5
    else:
        return 0
    
qq = train['GameWeather'].replace(np.nan, 'NAN').map(low).map(weather)

In [ ]:
qq.unique()

In [ ]:
train['GameWeather'] = qq

In [ ]:
train.iloc[0,:]

In [ ]:
trains = train.drop(ctd[2:], axis=1)

In [ ]:
trains = trains.drop(['GameId', 'PlayId'], axis=1)

In [ ]:
trains.iloc[5000,:]

In [ ]:
trains['TimeHandoff'] = trains['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
trains['TimeSnap'] = trains['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))

In [ ]:
trains['TimeDelta'] = trains.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)

In [ ]:
trains = trains.drop(['TimeHandoff', 'TimeSnap'], axis=1)

In [ ]:
train['FieldPosition'].replace(np.nan, 'NAN').unique()

In [ ]:
trains['HomeField'] = trains['FieldPosition'] == trains['HomeTeam']
trains['Field_eq_Possession'] = trains['FieldPosition'] == trains['PossessionTeam']

In [ ]:
trains.columns

In [ ]:
import lightgbm as lgb

In [3]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
train = trains.copy()

In [7]:
cat_features = []
for col in train.columns:
    if train[col].dtype =='object':
        cat_features.append(col)
        
train = train.drop(cat_features, axis=1)

In [8]:
cat_features

[]

In [9]:
train.fillna(-999, inplace=True)

In [10]:
players_col = []
for col in train.columns:
    if train[col][:22].std()!=0:
        players_col.append(col)

In [11]:
X_train = np.array(train[players_col]).reshape(-1, len(players_col)*22)


In [12]:
play_col = train.drop(players_col+['Yards'], axis=1).columns
X_play_col = np.zeros(shape=(X_train.shape[0], len(play_col)))
for i, col in enumerate(play_col):
    X_play_col[:, i] = train[col][::22]

In [13]:
X_train = np.concatenate([X_train, X_play_col], axis=1)
y_train = np.zeros(shape=(X_train.shape[0], 199))
for i,yard in enumerate(train['Yards'][::22]):
    y_train[i, yard+99:] = np.ones(shape=(1, 100-yard))

In [14]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [15]:
batch_size=64

In [18]:
class RAdam(keras.optimizers.Optimizer):
    """RAdam optimizer.
    # Arguments
        learning_rate: float >= 0. Learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
        decay: float >= 0. Learning rate decay over each update.
        weight_decay: float >= 0. Weight decay for each param.
        amsgrad: boolean. Whether to apply the AMSGrad variant of this
            algorithm from the paper "On the Convergence of Adam and
            Beyond".
        total_steps: int >= 0. Total number of training steps. Enable warmup by setting a positive value.
        warmup_proportion: 0 < warmup_proportion < 1. The proportion of increasing steps.
        min_lr: float >= 0. Minimum learning rate after warmup.
    # References
        - [Adam - A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980v8)
        - [On the Convergence of Adam and Beyond](https://openreview.net/forum?id=ryQu7f-RZ)
        - [On The Variance Of The Adaptive Learning Rate And Beyond](https://arxiv.org/pdf/1908.03265v1.pdf)
    """

    def __init__(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., weight_decay=0., amsgrad=False,
                 total_steps=0, warmup_proportion=0.1, min_lr=0., **kwargs):
        learning_rate = kwargs.pop('lr', learning_rate)
        super(RAdam, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.learning_rate = K.variable(learning_rate, name='learning_rate')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            self.weight_decay = K.variable(weight_decay, name='weight_decay')
            self.total_steps = K.variable(total_steps, name='total_steps')
            self.warmup_proportion = K.variable(warmup_proportion, name='warmup_proportion')
            self.min_lr = K.variable(min_lr, name='min_lr')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.initial_weight_decay = weight_decay
        self.initial_total_steps = total_steps
        self.amsgrad = amsgrad

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr

        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * K.cast(self.iterations, K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1

        if self.initial_total_steps > 0:
            warmup_steps = self.total_steps * self.warmup_proportion
            decay_steps = K.maximum(self.total_steps - warmup_steps, 1)
            decay_rate = (self.min_lr - lr) / decay_steps
            lr = K.switch(
                t <= warmup_steps,
                lr * (t / warmup_steps),
                lr + decay_rate * K.minimum(t - warmup_steps, decay_steps),
            )

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='m_' + str(i)) for (i, p) in enumerate(params)]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='v_' + str(i)) for (i, p) in enumerate(params)]

        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='vhat_' + str(i)) for (i, p) in enumerate(params)]
        else:
            vhats = [K.zeros(1, name='vhat_' + str(i)) for i in range(len(params))]

        self.weights = [self.iterations] + ms + vs + vhats

        beta_1_t = K.pow(self.beta_1, t)
        beta_2_t = K.pow(self.beta_2, t)

        sma_inf = 2.0 / (1.0 - self.beta_2) - 1.0
        sma_t = sma_inf - 2.0 * t * beta_2_t / (1.0 - beta_2_t)

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)

            m_corr_t = m_t / (1.0 - beta_1_t)
            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                v_corr_t = K.sqrt(vhat_t / (1.0 - beta_2_t))
                self.updates.append(K.update(vhat, vhat_t))
            else:
                v_corr_t = K.sqrt(v_t / (1.0 - beta_2_t))

            r_t = K.sqrt((sma_t - 4.0) / (sma_inf - 4.0) *
                         (sma_t - 2.0) / (sma_inf - 2.0) *
                         sma_inf / sma_t)

            p_t = K.switch(sma_t >= 5, r_t * m_corr_t / (v_corr_t + self.epsilon), m_corr_t)

            if self.initial_weight_decay > 0:
                p_t += self.weight_decay * p

            p_t = p - lr * p_t

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    @property
    def lr(self):
        return self.learning_rate

    @lr.setter
    def lr(self, learning_rate):
        self.learning_rate = learning_rate

    def get_config(self):
        config = {
            'learning_rate': float(K.get_value(self.learning_rate)),
            'beta_1': float(K.get_value(self.beta_1)),
            'beta_2': float(K.get_value(self.beta_2)),
            'decay': float(K.get_value(self.decay)),
            'weight_decay': float(K.get_value(self.weight_decay)),
            'epsilon': self.epsilon,
            'amsgrad': self.amsgrad,
            'total_steps': float(K.get_value(self.total_steps)),
            'warmup_proportion': float(K.get_value(self.warmup_proportion)),
            'min_lr': float(K.get_value(self.min_lr)),
        }
        base_config = super(RAdam, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [17]:
import keras

Using TensorFlow backend.


In [19]:
def crps(y_true, y_pred):
    return K.mean(K.square(y_true - K.cumsum(y_pred, axis=1)), axis=1)

In [20]:
def get_model():
    x = keras.layers.Input(shape=[X_train.shape[1]])
    fc1 = keras.layers.Dense(units=512, input_shape=[X_train.shape[1]])(x)
    act1 = keras.layers.PReLU()(fc1)
    bn1 = keras.layers.BatchNormalization()(act1)
    dp1 = keras.layers.Dropout(0.5)(bn1)
    concat1 = keras.layers.Concatenate()([x, dp1])
    fc2 = keras.layers.Dense(units=512)(concat1)
    act2 = keras.layers.PReLU()(fc2)
    bn2 = keras.layers.BatchNormalization()(act2)
    dp2 = keras.layers.Dropout(0.5)(bn2)
    concat2 = keras.layers.Concatenate()([concat1, dp2])
    output = keras.layers.Dense(units=199, activation='softmax')(concat2)
    model = keras.models.Model(inputs=[x], outputs=[output])
    return model


def train_model(X_train, y_train, X_val, y_val):
    model = get_model()
    model.compile(optimizer=RAdam(warmup_proportion=0.1, min_lr=1e-6), loss=crps)
    er = EarlyStopping(patience=15, min_delta=1e-4, restore_best_weights=True, monitor='val_loss')
    model.fit(X_train, y_train, epochs=100, callbacks=[er], validation_data=[X_val, y_val], batch_size=batch_size)
    return model

In [23]:
from sklearn.model_selection import RepeatedKFold

rkf = RepeatedKFold(n_splits=5, n_repeats=5)

models = []

for tr_idx, vl_idx in rkf.split(X_train, y_train):
    
    x_tr, y_tr = X_train[tr_idx], y_train[tr_idx]
    x_vl, y_vl = X_train[vl_idx], y_train[vl_idx]
    
    model = train_model(x_tr, y_tr, x_vl, y_vl)
    models.append(model)



Train on 18536 samples, validate on 4635 samples
Epoch 1/100


InternalError: 2 root error(s) found.
  (0) Internal: Blas GEMM launch failed : a.shape=(64, 390), b.shape=(390, 512), m=64, n=512, k=390
	 [[{{node dense_4/MatMul}}]]
	 [[loss/mul/_255]]
  (1) Internal: Blas GEMM launch failed : a.shape=(64, 390), b.shape=(390, 512), m=64, n=512, k=390
	 [[{{node dense_4/MatMul}}]]
0 successful operations.
0 derived errors ignored.

In [22]:
import keras
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.utils import plot_model
import keras.backend as K
import tensorflow as tf

In [ ]:
trains.to_csv('input/fe_train.csv', index=False)

In [ ]:
train

In [ ]:
train.to_csv('test.csv', index=False)

In [2]:
train = pd.read_csv('test.csv')